In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')

In [ ]:
train.head()

In [ ]:
train_diff_in = pd.get_dummies(train['difficulty'], dtype='float32')

In [ ]:
np_diff_in = train_diff_in.values

In [ ]:
np_diff_in.dtype

In [ ]:
row_num = train['ciphertext'].size
np_inputs = np.zeros((row_num, 300), dtype=np.float32)
np_inputs.dtype

In [ ]:
for i, e in train['ciphertext'].iteritems():
    #print(e)
    encoded = e.encode("ascii", errors='strict')
    lmao = np.frombuffer(encoded, dtype=np.byte)
    #if (lmao.shape[0] != 300):
    #    print("%d %d %d" % (lmao.shape[0], i, len(encoded)))
    np_inputs[i, :len(encoded)] = lmao

In [ ]:
np_inputs

def bytes_from_str(x):
    return x.encode("ascii", errors='strict')

type(bytes_from_str("lmao"))

is_this_input = train['ciphertext'].apply(bytes_from_str)

is_this_input.dtype

is_this_input.head()

this_is_input_val = is_this_input.values

this_is_input_val.shape

type(this_is_input_val[0])

lmao = train['ciphertext'][0]
type(lmao)

In [ ]:
from keras.models import Model
import keras.layers as KL

In [ ]:
import keras.backend as K

In [ ]:
# This returns a tensor
# inputs = KL.Input(shape=(300,), name="inputs")
inputs = KL.Input(shape=(1,300), name="inputs")
diff_in = KL.Input(shape=(4,), name="diff_in")

#reshaped = KL.Reshape(target_shape=(1,))(inputs)
conv_thing = KL.Conv1D(128, kernel_size=3, strides=2, data_format="channels_first", name='conv1')(inputs)
conv_thing = KL.Conv1D(32, kernel_size=3, strides=2, data_format="channels_first", name='conv2')(conv_thing)
conv_thing = KL.Conv1D(32, kernel_size=3, strides=2, data_format="channels_first", name='conv3')(conv_thing)
conv_thing = KL.Conv1D(32, kernel_size=3, strides=2, data_format="channels_first", name='conv4')(conv_thing)
conv_thing = KL.Conv1D(32, kernel_size=3, strides=2, data_format="channels_first", name='conv5')(conv_thing)
conv_thing = KL.Conv1D(32, kernel_size=3, strides=2, data_format="channels_first", name='conv6')(conv_thing)
conv_thing = KL.Conv1D(16, kernel_size=3, strides=2, data_format="channels_first", name='conv7')(conv_thing)
conv_thing =  KL.Lambda(lambda x: K.squeeze(x, axis=2))(conv_thing)

#x = KL.Dropout(0.2, name='drop0')(inputs)
x = KL.Concatenate(axis=1, name='concat0')([conv_thing, diff_in])#([x, diff_in])

# a layer instance is callable on a tensor, and returns a tensor
x = KL.Dense(10, activation='relu', name='dense0')(x)

d0 = x = KL.Dropout(0.2, name='drop1')(x)
x = KL.Dense(10, activation='relu', name='dense1')(x)
#x = KL.Concatenate(axis=1, name='concat1')([x, lol0])
x = KL.Dropout(0.2, name='drop2')(x)
x = KL.Dense(10, activation='relu', name='dense2')(x)
x = KL.Concatenate(axis=1, name='concat1')([x, d0])

d1 = x = KL.Dropout(0.2, name='drop3')(x)
x = KL.Dense(10, activation='relu', name='dense3')(x)
#x = KL.Concatenate(axis=1, name='concat2')([x, lol1])
x = KL.Dropout(0.2, name='drop4')(x)
x = KL.Dense(10, activation='relu', name='dense4')(x)
x = KL.Concatenate(axis=1, name='concat2')([x, d1])
x = KL.Dropout(0.2, name='drop6')(x)

predictions = KL.Dense(20, activation='softmax', name='dense5')(x)

# This creates a model that includes
# the Input layer and three Dense layers
in_stuff = [inputs, diff_in]
model = Model(inputs=in_stuff, outputs=predictions)
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit(data, labels)  # starts training

In [ ]:
model.summary()

model = Sequential()
model.add(KL.Dense(20, input_dim=304))
model.add(KL.Dropout(.1))
model.add(Activation('relu'))

x = lmao.encode(encoding='ascii', errors='strict') 

In [ ]:
np_inputs_norm = np_inputs / 127

In [ ]:
np_inputs_norm.shape

In [ ]:
np_diff_in.shape

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
label_thingy = pd.get_dummies(train['target'], dtype='float32')

In [ ]:
labels = label_thingy.values

In [ ]:
labels.shape

In [ ]:
!mkdir logs

In [ ]:
import keras.callbacks as KC

In [ ]:
checkpt = KC.ModelCheckpoint('logs/weights.{epoch:02d}-{acc:.2f}.hdf5', monitor='acc', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)

In [ ]:
model.fit([np_inputs_norm[:,np.newaxis,:], np_diff_in], labels, callbacks=[checkpt], epochs=5)  # starts training

In [ ]:
!ls

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model, to_file='model.png')

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import matplotlib.image as mpimg

img=mpimg.imread('model.png')

imgplot = plt.imshow(img)

from matplotlib.pyplot import figure


figure(figsize=(6,6))

In [ ]:

#There are several ways to post an image in Jupyter notebooks:

#via HTML:

from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "model.png")

https://stackoverflow.com/questions/32370281/how-to-embed-image-or-picture-in-jupyter-notebook-either-from-a-local-machine-o